Imports -- Donot Touch, Donot edit if you don't know what you are doing!

In [409]:
import time
import requests
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta

Variables -- Please edit based on your convenience

In [410]:
pincodes = [751015, 751030]   # Please provide 1 or more Pincodes
availability = 'stock'  # 'all' or 'stock'
fee_type = 'both'  # 'Paid' or 'Free' or "both"
age_limit = None  # 18 or 45 or None
start = None  # Time or None  | Format: '04:30PM' or None
end = None  # Time or None  | Format: '04:30PM' or None
lookup_start_date = date(2021, 5, 3)  # YYYY, m, dd
lookup_end_date = date(2021, 5, 20)  # YYYY, m, dd

Functions begin -- Please don't touch and don't edit if you don't know what you are doing!

In [411]:
delta = (lookup_end_date - lookup_start_date).days + 1
range_dates = [lookup_start_date + timedelta(days=day) for day in range(delta)]
date_range = [x.strftime('%d-%m-%Y') for x in range_dates]

Time Conversion

In [412]:
def convert_time(date_str):
  time_value = datetime.strptime(date_str, '%I:%M%p').time()
  return time_value

if start:
  start_time = datetime.strptime(start, '%I:%M%p').time()
else:
  start_time = start

if end:
  finish_time = datetime.strptime(end, '%I:%M%p').time()
else:
  finish_time = end

Function to get centres sorted by the most available ones first

In [413]:
def get_centers(read_json, fee_type: str, age_limit: int, start_time: datetime, finish_time: datetime, availability:str):
  """
  @param read_json: Json variable
  @availability: If vaccines are in stock
  @fee_type: Free or paid
  @age_limit: 18 or 45
  @start_time: Time for vaccine to start
  @finish_time: Time for vaccine to end
  """

  centers = pd.DataFrame(read_json.get('centers'))

  list_sessions = []
  for j, row in centers.iterrows():
    session = pd.DataFrame(row['sessions'][0])
    session['center_id'] = centers.loc[j, 'center_id']
    list_sessions.append(session)

  sessions = pd.concat(list_sessions, ignore_index=True)
  session_center = centers.merge(sessions, on='center_id')
  session_center.drop(columns=['sessions', 'session_id'], inplace=True)

  if availability == 'stock':
    session_center = session_center[session_center['available_capacity'] > 0]
  
  if fee_type == 'Paid':
    session_center = session_center[session_center['fee_type'] == 'Paid']
  elif fee_type == 'Free':
    session_center = session_center[session_center['fee_type'] == 'Free']
  
  if age_limit == 18:
    session_center = session_center[session_center['min_age_limit'] == 18]
  elif age_limit == 45:
    session_center = session_center[session_center['min_age_limit'] == 45]

  if not session_center.empty:
    session_center[['start', 'finish']] = session_center['slots'].str.split('-', expand=True)
    session_center['start'] = session_center['start'].apply(lambda x: convert_time(x))
    session_center['finish'] = session_center['finish'].apply(lambda x: convert_time(x))

    if start_time and finish_time:
      session_center = session_center[(session_center['start'] >= start_time) & (session_center['start'] <= finish_time)]
    elif start_time:
      session_center = session_center[session_center['start'] >= start_time]
    elif finish_time:
      session_center = session_center[session_center['start'] <= finish_time]
  
  session_center.reset_index(drop=True, inplace=True)
  return session_center

In [417]:
data_list = []

for date in date_range:
  for pincode in pincodes:
    response = requests.get(f"https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByPin?pincode={pincode}&date={date}")
    read_json = response.json()
    if 'Forbidden' in read_json.values():
      print(f'Message is forbidden, fetch is being blocked for PIN: {pincode} and Date: {date} - {read_json}')
    elif [] in read_json.values():
      print(f'Fetch successful, no vaccination centres available for PIN: {pincode} and Date: {date}  - {read_json}')
    else:
      data = get_centers(read_json, fee_type, age_limit, start_time, finish_time, availability)
      data_list.append(data)
data_set = pd.concat(data_list)
data_set.sort_values(['available_capacity', 'min_age_limit'], ascending=[False, True], inplace=True, ignore_index=True)
data_set


Message is forbidden, fetch is being blocked for PIN: 751015 and Date: 03-05-2021 - {'message': 'Forbidden'}
Message is forbidden, fetch is being blocked for PIN: 751015 and Date: 04-05-2021 - {'message': 'Forbidden'}
Message is forbidden, fetch is being blocked for PIN: 751030 and Date: 04-05-2021 - {'message': 'Forbidden'}
Message is forbidden, fetch is being blocked for PIN: 751015 and Date: 05-05-2021 - {'message': 'Forbidden'}
Message is forbidden, fetch is being blocked for PIN: 751030 and Date: 05-05-2021 - {'message': 'Forbidden'}
Message is forbidden, fetch is being blocked for PIN: 751015 and Date: 06-05-2021 - {'message': 'Forbidden'}
Message is forbidden, fetch is being blocked for PIN: 751030 and Date: 06-05-2021 - {'message': 'Forbidden'}
Message is forbidden, fetch is being blocked for PIN: 751030 and Date: 07-05-2021 - {'message': 'Forbidden'}
Message is forbidden, fetch is being blocked for PIN: 751015 and Date: 08-05-2021 - {'message': 'Forbidden'}
Message is forbidde

,center_id,name,state_name,district_name,block_name,pincode,lat,long,from,to,fee_type,date,available_capacity,min_age_limit,vaccine,slots,start,finish
0,564103,UPHC IRC VILLAGE,Odisha,Khurda,Bhubaneswar MC,751015,0,0,15:00:00,18:00:00,Free,07-05-2021,30,45,,03:00PM-05:00PM,15:00:00,17:00:00
1,564103,UPHC IRC VILLAGE,Odisha,Khurda,Bhubaneswar MC,751015,0,0,15:00:00,18:00:00,Free,07-05-2021,30,45,,05:00PM-06:00PM,17:00:00,18:00:00
